In [1]:
# !wget https://huggingface.co/datasets/mesolitica/translated-MNLI/resolve/main/translated-mnli-train.jsonl
# !wget https://huggingface.co/datasets/mesolitica/translated-MNLI/resolve/main/translated-mnli-dev_matched.jsonl

In [2]:
# !wget https://huggingface.co/datasets/mesolitica/translated-MNLI/resolve/main/anli.translated.jsonl

In [3]:
# !wget https://huggingface.co/datasets/mesolitica/translated-indonli/raw/main/train.jsonl
# !wget https://huggingface.co/datasets/mesolitica/translated-indonli/raw/main/validation.jsonl

In [4]:
from tqdm import tqdm
from glob import glob
import json
import random

random.random()

0.4137843164565038

In [5]:
labels = ['contradiction', 'entailment']

In [6]:
!head -n 1 translated-mnli-train.jsonl

{"annotator_labels": ["neutral"], "genre": "government", "gold_label": "neutral", "pairID": "31193n", "promptID": "31193", "sentence1": "Conceptually cream skimming has two basic dimensions - product and geography.", "sentence1_binary_parse": "( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )", "sentence1_parse": "(ROOT (S (NP (JJ Conceptually) (NN cream) (NN skimming)) (VP (VBZ has) (NP (NP (CD two) (JJ basic) (NNS dimensions)) (: -) (NP (NN product) (CC and) (NN geography)))) (. .)))", "sentence2": "Product and geography are what make cream skimming work. ", "sentence2_binary_parse": "( ( ( Product and ) geography ) ( ( are ( what ( make ( cream ( skimming work ) ) ) ) ) . ) )", "sentence2_parse": "(ROOT (S (NP (NN Product) (CC and) (NN geography)) (VP (VBP are) (SBAR (WHNP (WP what)) (S (VP (VBP make) (NP (NP (NN cream)) (VP (VBG skimming) (NP (NN work)))))))) (. .)))", "translate": ["Skim krim konseptual mempu

In [7]:
!head -n 1 anli.translated.jsonl

{"uid": "2093cfb3-a15f-4282-81e3-0cb793ffd0d7", "premise": "TOKYO, Dec 18 (Reuters) - Japan\u2019s Shionogi & Co said on Tuesday that it has applied to health regulators in the United States, Canada and Europe for approval of its HIV drug Dolutegravir. Shionogi developed Dolutegravir with a Viiv Healthcare, an AIDS drug joint venture between GlaxoSmithKline and Pfizer, in exchange for its rights to the drug.", "hypothesis": "The article was written on December 18th.", "label": 0, "reason": "TOKYO, Dec 18 (Reuters) is when the article was written as it states in the first words of the sentence", "premise_ms": "TOKYO, 18 Dis (Reuters) - Shionogi & Co Jepun berkata pada hari Selasa bahawa ia telah memohon kepada pengawal selia kesihatan di Amerika Syarikat, Kanada dan Eropah untuk kelulusan ubat HIVnya Dolutegravir. Shionogi membangunkan Dolutegravir dengan Viiv Healthcare, sebuah usaha sama ubat AIDS antara GlaxoSmithKline dan Pfizer, sebagai pertukaran untuk haknya terhadap ubat tersebu

In [8]:
anli = {
    0: 1,
    2: 0,
}

In [9]:
with open('train.jsonl') as fopen:
    for l in fopen:
        data = json.loads(l)
        break

In [10]:
data

{'premise': 'Presiden Joko Widodo (Jokowi) menyampaikan prediksi bahwa wabah virus Corona (COVID-19) di Indonesia akan selesai akhir tahun ini.',
 'hypothesis': 'Prediksi akhir wabah tidak disampaikan Jokowi.',
 'label': 2,
 'premise_ms': 'Presiden Joko Widodo (Jokowi) menyampaikan prediksi bahwa wabah virus Corona (COVID-19) di Indonesia akan selesai akhir tahun ini.',
 'hypothesis_ms': '"Ramalan akhir wabak tidak dikemukakan oleh Jokowi."'}

In [11]:
with open('train.json', 'w') as fopen_jsonl:
    with open('train.jsonl') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            label = data['label']
            label = anli.get(label)
            if label is None:
                continue
            
            sent1 = data['premise_ms'].strip()
            sent2 = data['hypothesis_ms'].strip()
            
            left = f'ayat1: {sent1} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
    with open('anli.translated.jsonl') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            label = data['label']
            label = anli.get(label)
            if label is None:
                continue
                
            sent1 = data['premise'].strip()
            sent2 = data['hypothesis'].strip()
            
            sent1_ms = data['premise_ms'].strip()
            sent2_ms = data['hypothesis_ms'].strip()
            
            left = f'ayat1: {sent1} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1_ms} ayat2: {sent2_ms}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            if random.random() > 0.8:
            
                left = f'ayat1: {sent1} ayat2: {sent2_ms}'
                d = {"src": left, "label": label}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            if random.random() > 0.8:
            
                left = f'ayat1: {sent1_ms} ayat2: {sent2}'
                d = {"src": left, "label": label}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
            
    with open('translated-mnli-train.jsonl') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            label = data['gold_label']
            if label not in labels:
                continue
            if label == '-':
                continue
            
            label = labels.index(label)
            sent1 = data['sentence1'].strip()
            sent2 = data['sentence2'].strip()
            
            sent1_ms = data['translate'][0].strip()
            sent2_ms = data['translate'][1].strip()
            
            left = f'ayat1: {sent1} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1_ms} ayat2: {sent2_ms}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            if random.random() > 0.8:
            
                left = f'ayat1: {sent1} ayat2: {sent2_ms}'
                d = {"src": left, "label": label}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                
            if random.random() > 0.8:
            
                left = f'ayat1: {sent1_ms} ayat2: {sent2}'
                d = {"src": left, "label": label}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
    
    with open('sentiment.json') as fopen:
        data = json.load(fopen)
        for i in range(len(data['X1'])):
            l = data['X1'][i]
            r = data['X2'][i]
            left = f'ayat1: {l} ayat2: {r}'
            d = {"src": left, "label": data['Y'][i]}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
    with open('topics.json') as fopen:
        data = json.load(fopen)
        for i in range(len(data['X1'])):
            l = data['X1'][i]
            r = data['X2'][i]
            left = f'ayat1: {l} ayat2: {r}'
            d = {"src": left, "label": data['Y'][i]}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
    with open('positive-toxicity.json') as fopen:
        data = json.load(fopen)
        for i in range(len(data['X1'])):
            l = data['X1'][i]
            r = data['X2'][i]
            left = f'ayat1: {l} ayat2: {r}'
            d = {"src": left, "label": data['Y'][i]}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
    with open('negative-toxicity.json') as fopen:
        data = json.load(fopen)
        for i in range(len(data['X1'])):
            l = data['X1'][i]
            r = data['X2'][i]
            left = f'ayat1: {l} ayat2: {r}'
            d = {"src": left, "label": data['Y'][i]}
            fopen_jsonl.write(f'{json.dumps(d)}\n')

10330it [00:00, 287687.40it/s]
100459it [00:00, 161176.28it/s]
392702it [00:02, 143846.66it/s]


In [12]:
!wc -l train.json

1093561 train.json


In [13]:
!shuf train.json > shuffled-train.json

In [14]:
with open('test.json', 'w') as fopen_jsonl:
    with open('translated-mnli-dev_matched.jsonl') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            label = data['gold_label']
            if label not in labels:
                continue
            if label == '-':
                continue
            
            label = labels.index(label)
            sent1 = data['sentence1'].strip()
            sent2 = data['sentence2'].strip()
            
            sent1_ms = data['translate'][0].strip()
            sent2_ms = data['translate'][1].strip()
            
            left = f'ayat1: {sent1} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1_ms} ayat2: {sent2_ms}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            if random.random() > 0.8:
            
                left = f'ayat1: {sent1} ayat2: {sent2_ms}'
                d = {"src": left, "label": label}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                
            if random.random() > 0.8:
            
                left = f'ayat1: {sent1_ms} ayat2: {sent2}'
                d = {"src": left, "label": label}
                fopen_jsonl.write(f'{json.dumps(d)}\n')

10000it [00:00, 123856.58it/s]


In [15]:
!shuf test.json > shuffled-test.json

In [16]:
!head -n 1000 shuffled-test.json > test-1k.json

In [17]:
!wc -l shuffled-test.json

16037 shuffled-test.json


In [18]:
!head -n 10 shuffled-test.json

{"src": "ayat1: Tidak ada Tuhan di sini sekarang, kata suara raksasa di hadapan mereka. ayat2: A monster spoke to them.", "label": 1}
{"src": "ayat1: Ini adalah ketepuan dan 125 keping urutan berjalan Enhanced Carrier Route mel volume pada tahun 1996. ayat2: Laluan Pengangkut Enhanced diperkenalkan sebelum tahun 1997.", "label": 1}
{"src": "ayat1: Untuk mencapai mana-mana tiga Carbet jatuh, anda mesti terus berjalan selepas jalan-jalan berakhir selama 20 minit, 30 minit, atau dua jam masing-masing. ayat2: Terdapat tiga laluan ke tiga Carbet jatuh, masing-masing panjang yang berbeza dan semuanya berterusan selepas jalan seolah-olah berakhir.", "label": 1}
{"src": "ayat1: Dia mempunyai dua fikiran, satu gembira dalam kedamaian kampung ini. ayat2: Kampung ini penuh dengan keganasan.", "label": 0}
{"src": "ayat1: but uh i've always enjoyed uh the train and you know fooling with it and all ayat2: Saya selalu suka kereta api dan bermain-main dengannya.", "label": 1}
{"src": "ayat1: Hampir la